# Data preprocessing

With least squares we can find the optimal w to minimize the loss (mean square error) in our train data of a linear regression model. We have accomplished a 75% of accuracy in test data this way. Since least squares finds the optimal w, we already have the "best" model for our data, if we want to improve our model, we have to "improve" our data by cleaning/handling it.

In this notebook we try to find a proper way to preprocess our data in order to achieve higher accuracy in linear regression models.

In [1]:
import numpy  as np
import pandas as pd

In [2]:
TRAIN_DATA_PATH = '../data/train.csv'
TEST_DATA_PATH  = '../data/test.csv'
OUTPUT_PATH     = '../data/output.csv'

In [3]:
df_train = pd.read_csv(TRAIN_DATA_PATH, index_col = 0)
df_test  = pd.read_csv(TEST_DATA_PATH,  index_col = 0)

In [4]:
# List all variables and types
df_train.dtypes

Prediction                      object
DER_mass_MMC                   float64
DER_mass_transverse_met_lep    float64
DER_mass_vis                   float64
DER_pt_h                       float64
DER_deltaeta_jet_jet           float64
DER_mass_jet_jet               float64
DER_prodeta_jet_jet            float64
DER_deltar_tau_lep             float64
DER_pt_tot                     float64
DER_sum_pt                     float64
DER_pt_ratio_lep_tau           float64
DER_met_phi_centrality         float64
DER_lep_eta_centrality         float64
PRI_tau_pt                     float64
PRI_tau_eta                    float64
PRI_tau_phi                    float64
PRI_lep_pt                     float64
PRI_lep_eta                    float64
PRI_lep_phi                    float64
PRI_met                        float64
PRI_met_phi                    float64
PRI_met_sumet                  float64
PRI_jet_num                      int64
PRI_jet_leading_pt             float64
PRI_jet_leading_eta      

In [5]:
# Brief overview of our train data
df_train.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,-49.023079,49.239819,81.181982,57.895962,-708.420675,-601.237051,-709.356603,2.373100,18.917332,158.432217,...,-0.010119,209.797178,0.979176,-348.329567,-399.254314,-399.259788,-692.381204,-709.121609,-709.118631,73.064591
std,406.345647,35.344886,40.828691,63.655682,454.480565,657.972302,453.019877,0.782911,22.273494,115.706115,...,1.812223,126.499506,0.977426,532.962789,489.338286,489.333883,479.875496,453.384624,453.389017,98.015662
min,-999.000000,0.000000,6.329000,0.000000,-999.000000,-999.000000,-999.000000,0.208000,0.000000,46.104000,...,-3.142000,13.678000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,78.100750,19.241000,59.388750,14.068750,-999.000000,-999.000000,-999.000000,1.810000,2.841000,77.550000,...,-1.575000,123.017500,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,105.012000,46.524000,73.752000,38.467500,-999.000000,-999.000000,-999.000000,2.491500,12.315500,120.664500,...,-0.024000,179.739000,1.000000,38.960000,-1.872000,-2.093000,-999.000000,-999.000000,-999.000000,40.512500
75%,130.606250,73.598000,92.259000,79.169000,0.490000,83.446000,-4.593000,2.961000,27.591000,200.478250,...,1.561000,263.379250,2.000000,75.349000,0.433000,0.503000,33.703000,-2.457000,-2.275000,109.933750
max,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,1852.462000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


As we are trying to improve as much as possible our **linear regression** model, outliers can affect significantly the precission we can obtain. If we don't remove them our model would try to fit the outliers and this could cause a deviation in the other and "more correct" training data. Other (equally important) issue we have to solve are the huge amount of missing values (those with -999) in certain variables. As seen in above table some of the variables have more than the half of the data with missing values. We have to decide if these variables can be dropped or if they contribute to the model accuarcy. In other words, are they providing noise or information? Is there any way we can manipulate these missing values to minimize the noise and maximize the information?

Therefore the first issues we have to solve are:
* **Outliers (OL)**
* **Missing values (MV)**

As an important final remark we'll add that even though the decissions about how to handle these issues are going to be taken by analyzing the effects caused in the training data model we have to always keep an eye in the test data and be sure that it's also having the same properties.

In [6]:
# Brief overview of our test data
df_test.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.00000,568238.000000,...,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000,568238.000000
mean,-48.729241,49.258387,81.122338,57.829094,-707.441800,-599.731058,-708.384205,2.374211,18.99262,158.668286,...,-0.007981,209.957809,0.980251,-348.946261,-399.886426,-399.899229,-691.293904,-708.143299,-708.146201,73.267629
std,406.018702,35.393465,40.474035,63.304450,454.931763,659.054554,453.464437,0.779978,21.76045,116.258246,...,1.812916,126.956060,0.979394,533.156405,489.468578,489.458204,480.450337,453.837535,453.832741,98.470522
min,-999.000000,0.000000,6.810000,0.000000,-999.000000,-999.000000,-999.000000,0.237000,0.00000,46.103000,...,-3.142000,13.847000,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
25%,78.191000,19.330000,59.425000,14.202250,-999.000000,-999.000000,-999.000000,1.815000,2.83800,77.463000,...,-1.574000,122.972250,0.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,0.000000
50%,105.080000,46.467000,73.740000,38.472000,-999.000000,-999.000000,-999.000000,2.492000,12.41300,120.666000,...,-0.016000,179.940000,1.000000,38.968000,-1.862000,-2.110000,-999.000000,-999.000000,-999.000000,40.504000
75%,130.775500,73.630000,92.162750,79.256000,0.503000,84.305500,-4.532000,2.962000,27.65100,201.073000,...,1.559000,264.024750,2.000000,75.520000,0.431000,0.483000,33.838000,-2.427000,-2.260000,110.566500
max,1949.261000,968.669000,1264.965000,1337.187000,8.724000,4794.827000,17.650000,5.751000,759.36300,2079.162000,...,3.142000,2190.275000,3.000000,1163.439000,4.500000,3.142000,817.801000,4.500000,3.142000,1860.175000


Given the information of the following table we can be calm since (at least in general terms) train and test data seems to be very similarly distributed.

In [7]:
# Difference of descriptive statistics in train and test data
(df_train.describe() - df_test.describe()).drop(['count'], axis=0)

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
mean,-0.293839,-0.018568,0.059644,0.066868,-0.978876,-1.505992,-0.972398,-0.001111,-0.075288,-0.236069,...,-0.002138,-0.160631,-0.001075,0.616694,0.632112,0.639441,-1.087300,-0.978310,-0.972430,-0.203037
std,0.326944,-0.048579,0.354655,0.351232,-0.451198,-1.082251,-0.444561,0.002933,0.513043,-0.552130,...,-0.000693,-0.456555,-0.001968,-0.193616,-0.130292,-0.124321,-0.574841,-0.452911,-0.443723,-0.454860
min,0.000000,0.000000,-0.481000,0.000000,0.000000,0.000000,0.000000,-0.029000,0.000000,0.001000,...,0.000000,-0.169000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.090250,-0.089000,-0.036250,-0.133500,0.000000,0.000000,0.000000,-0.005000,0.003000,0.087000,...,-0.001000,0.045250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-0.068000,0.057000,0.012000,-0.004500,0.000000,0.000000,0.000000,-0.000500,-0.097500,-0.001500,...,-0.008000,-0.201000,0.000000,-0.008000,-0.010000,0.017000,0.000000,0.000000,0.000000,0.008500
75%,-0.169250,-0.032000,0.096250,-0.087000,-0.013000,-0.859500,-0.061000,-0.001000,-0.060000,-0.594750,...,0.002000,-0.645500,0.000000,-0.171000,0.002000,0.020000,-0.135000,-0.030000,-0.015000,-0.632750
max,-757.235000,-278.594000,84.386000,1497.812000,-0.221000,180.152000,-0.960000,-0.067000,2075.636000,-226.700000,...,0.000000,-186.299000,0.000000,-42.866000,-0.001000,-0.001000,-96.345000,0.000000,0.000000,-226.742000


## 1.- Missing values

It's very important to remark that the **order** (what to handle first, MV or OL?) **is important**. We have decided here that first we are trying to handle the MV problem because if not, we could confuse outliers with data with several missing values (since MV entries are -999, far from the non missing values range in most cases).

#### 1.1 Removing MV

Dead the dog, the rabies is gone. We can dismiss every feature with missing values entries. This option might not be a good choice since it seems that we are losing plenty of information. However, it could be the case that these features to be providing more noise to our model than information. It's free to try so we have no excuse for not doing it.

In [8]:
# Features including missing values
features_with_MV = ['DER_mass_MMC','DER_deltaeta_jet_jet','DER_lep_eta_centrality','DER_mass_jet_jet','DER_prodeta_jet_jet','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi']

In [9]:
# Dismiss features with missing values
df_train_without_MV_columns = df_train.drop(features_with_MV, axis=1)
df_test_without_MV_columns  = df_test.drop( features_with_MV, axis=1)

#df_train_without_MV_columns.describe()
#df_test_without_MV_columns.describe()  # <- Important to check!

With the previous approach we are losing the information given by the deleted features in the case where they have a valid measurement. We should analize the following: 

* Was this data dropped correlated with the predicted variable? For example, could be the case of these features not being able to get a valid measurement when a Boson appears, we would be losing crucial information!
* How affects the suppression of this features to the data distribution? 

We can also try to analyze the data when dropping points with MV, however, with this approach we would be training our model without noise and therefore we can't predict how it will behave whith test data entries with missing values. However, if we are lucky we can get an incredible accuracy with this model for data without MV and for sure we can use it in our final model.

In [10]:
# Dismiss every point with a missing value
df_train_without_MV_points  = df_train[(df_train.DER_mass_MMC != -999) & (df_train.DER_deltaeta_jet_jet!=-999) & (df_train.DER_lep_eta_centrality!=-999) & (df_train.DER_mass_jet_jet!=-999) & (df_train.DER_prodeta_jet_jet!=-999) & (df_train.PRI_jet_leading_pt!=-999) & (df_train.PRI_jet_leading_eta!=-999) & (df_train.PRI_jet_leading_phi!=-999) & (df_train.PRI_jet_subleading_pt!=-999) & (df_train.PRI_jet_subleading_eta!=-999) & (df_train.PRI_jet_subleading_phi!=-999)]
df_test_without_MV_points  = df_test[(df_test.DER_mass_MMC != -999)  & (df_test.DER_deltaeta_jet_jet!=-999) & (df_test.DER_lep_eta_centrality!=-999) & (df_test.DER_mass_jet_jet!=-999) & (df_test.DER_prodeta_jet_jet!=-999) & (df_test.PRI_jet_leading_pt!=-999) & (df_test.PRI_jet_leading_eta!=-999) & (df_test.PRI_jet_leading_phi!=-999) & (df_test.PRI_jet_subleading_pt!=-999) & (df_test.PRI_jet_subleading_eta!=-999) & (df_test.PRI_jet_subleading_phi!=-999)]

#df_train_without_MV_points.describe()
#df_test_without_MV_points.describe()  # <- Important to check!

Removing points with MV of course is affecting drastically the data distribution in features with MV but, how is it affecting the other features? We can get a general idea again by analyzing the difference in the metrics of our data distribution.

In [11]:
# Show difference in descriptive statistics for original and without MV points distributions
#(df_train.describe() - df_train_without_MV_points.describe()).drop(features_with_MV, axis=1).drop(['count'], axis=0)
#(df_test.describe()  - df_test_without_MV_points.describe( )).drop(features_with_MV, axis=1).drop(['count'], axis=0)

#### 1.2 Replacing MV

Non valid measurements take the value -999 (totally arbitrary). This happens both in features which can take only positive values (therefore of course MV entries creates outliers in this dimension). 

For this purpose it totally make sense trying to replace these MV measurements for one neutral and more aseptic value so their impact in the decissionmaking is smaller. We can try to replace it by the average (or the mode in discrete and short range variables) of the feature valid measurements and, again since it's also free, we can try to replace it by a deviated but more neutral value (f.e. 0 for positive features). 

In [12]:
# features_with_MV = ['DER_mass_MMC','DER_deltaeta_jet_jet','DER_lep_eta_centrality','DER_mass_jet_jet','DER_prodeta_jet_jet','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi']

# Features taking also negative values:
# 'DER_prodeta_jet_jet'
# 'PRI_jet_leading_eta'
# 'PRI_jet_leading_phi'
# 'PRI_jet_subleading_eta'
# 'PRI_jet_subleading_phi'

In [13]:
# Replace MV with 0 (we have to copy df_train and df_test before!)
df_train_MV_replaced_0 = df_train
df_train_MV_replaced_0 = df_train_MV_replaced_0[features_with_MV].replace(-999,0)
df_test_MV_replaced_0  = df_test
df_test_MV_replaced_0  = df_test_MV_replaced_0[ features_with_MV].replace(-999,0)

# df_train_MV_replaced_0.describe()
# df_test_MV_replaced_0.describe()

In [14]:
# Replace MV with avg in train data
DER_mass_MMC_nonMV_avg           = df_train[df_train.DER_mass_MMC           != -999]['DER_mass_MMC'          ].mean()
DER_deltaeta_jet_jet_nonMV_avg   = df_train[df_train.DER_deltaeta_jet_jet   != -999]['DER_deltaeta_jet_jet'  ].mean()
DER_lep_eta_centrality_nonMV_avg = df_train[df_train.DER_lep_eta_centrality != -999]['DER_lep_eta_centrality'].mean()
DER_mass_jet_jet_nonMV_avg       = df_train[df_train.DER_mass_jet_jet       != -999]['DER_mass_jet_jet'      ].mean()
DER_prodeta_jet_jet_nonMV_avg    = df_train[df_train.DER_prodeta_jet_jet    != -999]['DER_prodeta_jet_jet'   ].mean()
PRI_jet_leading_pt_nonMV_avg     = df_train[df_train.PRI_jet_leading_pt     != -999]['PRI_jet_leading_pt'    ].mean()
PRI_jet_leading_eta_nonMV_avg    = df_train[df_train.PRI_jet_leading_eta    != -999]['PRI_jet_leading_eta'   ].mean()
PRI_jet_leading_phi_nonMV_avg    = df_train[df_train.PRI_jet_leading_phi    != -999]['PRI_jet_leading_phi'   ].mean()
PRI_jet_subleading_pt_nonMV_avg  = df_train[df_train.PRI_jet_subleading_pt  != -999]['PRI_jet_subleading_pt' ].mean()
PRI_jet_subleading_eta_nonMV_avg = df_train[df_train.PRI_jet_subleading_eta != -999]['PRI_jet_subleading_eta'].mean() 
PRI_jet_subleading_phi_nonMV_avg = df_train[df_train.PRI_jet_subleading_phi != -999]['PRI_jet_subleading_phi'].mean()

df_train_MV_replaced_avg = df_train.copy()         # IMPORTANT! .copy()
df_train_MV_replaced_avg['DER_mass_MMC']           = df_train_MV_replaced_avg['DER_mass_MMC'          ].replace(-999,DER_mass_MMC_nonMV_avg)
df_train_MV_replaced_avg['DER_deltaeta_jet_jet']   = df_train_MV_replaced_avg['DER_deltaeta_jet_jet'  ].replace(-999,DER_deltaeta_jet_jet_nonMV_avg)
df_train_MV_replaced_avg['DER_lep_eta_centrality'] = df_train_MV_replaced_avg['DER_lep_eta_centrality'].replace(-999,DER_lep_eta_centrality_nonMV_avg)
df_train_MV_replaced_avg['DER_mass_jet_jet']       = df_train_MV_replaced_avg['DER_mass_jet_jet'      ].replace(-999,DER_mass_jet_jet_nonMV_avg)
df_train_MV_replaced_avg['DER_prodeta_jet_jet']    = df_train_MV_replaced_avg['DER_prodeta_jet_jet'   ].replace(-999,DER_prodeta_jet_jet_nonMV_avg)
df_train_MV_replaced_avg['PRI_jet_leading_pt']     = df_train_MV_replaced_avg['PRI_jet_leading_pt'    ].replace(-999,PRI_jet_leading_pt_nonMV_avg)
df_train_MV_replaced_avg['PRI_jet_leading_eta']    = df_train_MV_replaced_avg['PRI_jet_leading_eta'   ].replace(-999,PRI_jet_leading_eta_nonMV_avg)
df_train_MV_replaced_avg['PRI_jet_leading_phi']    = df_train_MV_replaced_avg['PRI_jet_leading_phi'   ].replace(-999,PRI_jet_leading_phi_nonMV_avg)
df_train_MV_replaced_avg['PRI_jet_subleading_pt']  = df_train_MV_replaced_avg['PRI_jet_subleading_pt' ].replace(-999,PRI_jet_subleading_pt_nonMV_avg)
df_train_MV_replaced_avg['PRI_jet_subleading_eta'] = df_train_MV_replaced_avg['PRI_jet_subleading_eta'].replace(-999,PRI_jet_subleading_eta_nonMV_avg)
df_train_MV_replaced_avg['PRI_jet_subleading_phi'] = df_train_MV_replaced_avg['PRI_jet_subleading_phi'].replace(-999,PRI_jet_subleading_phi_nonMV_avg)

In [15]:
# This function generalizes the behaviour of the cell above
# WARNING! Modifies the input parameter df
def replace_MV_by_average(df, features, MV_value):
    for feature in features:
        df[feature] = df[feature].replace(MV_value, (df[df[feature] != MV_value][feature].mean()))

In [16]:
# Replace MV with avg in test data (1, using newly function)
df_test_MV_replaced_avg = df_test.copy()
replace_MV_by_average(df_test_MV_replaced_avg, features_with_MV, -999)

In [17]:
# Replace MV with avg in test data (2, ad hoc)
DER_mass_MMC_nonMV_avg           = df_test[df_test.DER_mass_MMC           != -999]['DER_mass_MMC'          ].mean()
DER_deltaeta_jet_jet_nonMV_avg   = df_test[df_test.DER_deltaeta_jet_jet   != -999]['DER_deltaeta_jet_jet'  ].mean()
DER_lep_eta_centrality_nonMV_avg = df_test[df_test.DER_lep_eta_centrality != -999]['DER_lep_eta_centrality'].mean()
DER_mass_jet_jet_nonMV_avg       = df_test[df_test.DER_mass_jet_jet       != -999]['DER_mass_jet_jet'      ].mean()
DER_prodeta_jet_jet_nonMV_avg    = df_test[df_test.DER_prodeta_jet_jet    != -999]['DER_prodeta_jet_jet'   ].mean()
PRI_jet_leading_pt_nonMV_avg     = df_test[df_test.PRI_jet_leading_pt     != -999]['PRI_jet_leading_pt'    ].mean()
PRI_jet_leading_eta_nonMV_avg    = df_test[df_test.PRI_jet_leading_eta    != -999]['PRI_jet_leading_eta'   ].mean()
PRI_jet_leading_phi_nonMV_avg    = df_test[df_test.PRI_jet_leading_phi    != -999]['PRI_jet_leading_phi'   ].mean()
PRI_jet_subleading_pt_nonMV_avg  = df_test[df_test.PRI_jet_subleading_pt  != -999]['PRI_jet_subleading_pt' ].mean()
PRI_jet_subleading_eta_nonMV_avg = df_test[df_test.PRI_jet_subleading_eta != -999]['PRI_jet_subleading_eta'].mean() 
PRI_jet_subleading_phi_nonMV_avg = df_test[df_test.PRI_jet_subleading_phi != -999]['PRI_jet_subleading_phi'].mean()

df_test_MV_replaced_avg1 = df_test.copy()          # IMPORTANT! .copy()
df_test_MV_replaced_avg1['DER_mass_MMC']           = df_test_MV_replaced_avg1['DER_mass_MMC'          ].replace(-999,DER_mass_MMC_nonMV_avg)
df_test_MV_replaced_avg1['DER_deltaeta_jet_jet']   = df_test_MV_replaced_avg1['DER_deltaeta_jet_jet'  ].replace(-999,DER_deltaeta_jet_jet_nonMV_avg)
df_test_MV_replaced_avg1['DER_lep_eta_centrality'] = df_test_MV_replaced_avg1['DER_lep_eta_centrality'].replace(-999,DER_lep_eta_centrality_nonMV_avg)
df_test_MV_replaced_avg1['DER_mass_jet_jet']       = df_test_MV_replaced_avg1['DER_mass_jet_jet'      ].replace(-999,DER_mass_jet_jet_nonMV_avg)
df_test_MV_replaced_avg1['DER_prodeta_jet_jet']    = df_test_MV_replaced_avg1['DER_prodeta_jet_jet'   ].replace(-999,DER_prodeta_jet_jet_nonMV_avg)
df_test_MV_replaced_avg1['PRI_jet_leading_pt']     = df_test_MV_replaced_avg1['PRI_jet_leading_pt'    ].replace(-999,PRI_jet_leading_pt_nonMV_avg)
df_test_MV_replaced_avg1['PRI_jet_leading_eta']    = df_test_MV_replaced_avg1['PRI_jet_leading_eta'   ].replace(-999,PRI_jet_leading_eta_nonMV_avg)
df_test_MV_replaced_avg1['PRI_jet_leading_phi']    = df_test_MV_replaced_avg1['PRI_jet_leading_phi'   ].replace(-999,PRI_jet_leading_phi_nonMV_avg)
df_test_MV_replaced_avg1['PRI_jet_subleading_pt']  = df_test_MV_replaced_avg1['PRI_jet_subleading_pt' ].replace(-999,PRI_jet_subleading_pt_nonMV_avg)
df_test_MV_replaced_avg1['PRI_jet_subleading_eta'] = df_test_MV_replaced_avg1['PRI_jet_subleading_eta'].replace(-999,PRI_jet_subleading_eta_nonMV_avg)
df_test_MV_replaced_avg1['PRI_jet_subleading_phi'] = df_test_MV_replaced_avg1['PRI_jet_subleading_phi'].replace(-999,PRI_jet_subleading_phi_nonMV_avg)

In [18]:
# df_train_MV_replaced_avg.describe()
# df_test_MV_replaced_avg.describe()

# To check function 'replace_MV_by_average' correctness
# df_test_MV_replaced_avg.describe() - df_test_MV_replaced_avg1.describe()

#### 1.3 Combination of both

We have features with a lot of MV and other with just a few of them. It makes sense then trying to keep features with only few MV and remove those which are suppose to add more noise than information. We are saying then that removing all features with at least one MV is a really drastic approach and loses a lot of information but also replacing MV by the average of valid measurements in features with more than 70% of MV is whiten noise.

In this section we are trying to obtain a more intelligent approach for data preprocessing based on these ideas.

In [19]:
# Get for every feature with MV their frequency in train and test data
train_MV_freqs = []
test_MV_freqs  = []
N_train = df_train.shape[0]
N_tests = df_test.shape[ 0]

for feature in features_with_MV:
    train_MV_freqs.append((df_train[df_train[feature]==-999]).shape[0]/N_train)
    test_MV_freqs.append( (df_test[ df_test[ feature]==-999]).shape[0]/N_tests)

features_with_MV_freq = pd.DataFrame({'MV_freq_train':train_MV_freqs, 'MV_freq_test':test_MV_freqs},features_with_MV)
features_with_MV_freq

,MV_freq_train,MV_freq_test
DER_mass_MMC,0.152456,0.152204
DER_deltaeta_jet_jet,0.709828,0.708851
DER_lep_eta_centrality,0.709828,0.708851
DER_mass_jet_jet,0.709828,0.708851
DER_prodeta_jet_jet,0.709828,0.708851
PRI_jet_leading_pt,0.399652,0.400286
PRI_jet_leading_eta,0.399652,0.400286
PRI_jet_leading_phi,0.399652,0.400286
PRI_jet_subleading_pt,0.709828,0.708851
PRI_jet_subleading_eta,0.709828,0.708851


Seeing the frequencies of the missing values in the different features we can decide which is more reasonable to keep. We have to be aware however that only by knowing the frequence of the MV we can't take a totally well-funded decission. There could be underlying relations between the desired prediction and one/some of these features. However, a priori it's reasonable to suppose that the huge amount of noise in these features is penalizing our model.

In [20]:
features_with_high_MV_freq       = ['DER_deltaeta_jet_jet','DER_lep_eta_centrality','DER_mass_jet_jet','DER_prodeta_jet_jet','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi']
features_with_reasonable_MV_freq = ['DER_mass_MMC','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi']

In [21]:
df_train_MV_cleaned = df_train.copy()
df_train_MV_cleaned = df_train_MV_cleaned.drop(features_with_high_MV_freq, axis=1)
replace_MV_by_average(df_train_MV_cleaned, features_with_reasonable_MV_freq, -999)

df_test_MV_cleaned = df_test.copy()
df_test_MV_cleaned = df_test_MV_cleaned.drop(features_with_high_MV_freq, axis=1)
replace_MV_by_average(df_test_MV_cleaned, features_with_reasonable_MV_freq, -999)

In [22]:
# df_train_MV_cleaned.describe()
# df_test_MV_cleaned.describe()

## 2.- Outliers

As we said, linear regression model is determined by w and w is the coefficients vector which minimizes the loss, this is, the average square error in every point. Having outliers makes us find an w which tries to suit these points also (and since we are using **square** error, giving it quite importance) even by reducing the precission in more correctly measured/common data points. 

In this section we'll try to remove outliers in our train dataset so we can find an w which fits better our model.

In [23]:
df_train_MV_cleaned.describe().drop(['count'])

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,PRI_tau_pt,...,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_all_pt
mean,121.858528,49.239819,81.181982,57.895962,2.373100,18.917332,158.432217,1.437609,-0.128305,38.707419,...,-0.019507,0.043543,41.717235,-0.010119,209.797178,0.979176,84.822105,-0.003275,-0.012393,73.064591
std,52.749898,35.344886,40.828691,63.655682,0.782911,22.273494,115.706115,0.844743,1.193585,22.412081,...,1.264982,1.816611,32.894693,1.812223,126.499506,0.977426,47.002359,1.382702,1.405048,98.015662
min,9.044000,0.000000,6.329000,0.000000,0.208000,0.000000,46.104000,0.047000,-1.414000,20.000000,...,-2.505000,-3.142000,0.109000,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,0.000000
25%,95.665000,19.241000,59.388750,14.068750,1.810000,2.841000,77.550000,0.883000,-1.371000,24.591750,...,-1.014000,-1.522000,21.398000,-1.575000,123.017500,0.000000,57.439000,-0.433000,-0.556000,0.000000
50%,119.958000,46.524000,73.752000,38.467500,2.491500,12.315500,120.664500,1.280000,-0.356000,31.804000,...,-0.045000,0.086000,34.802000,-0.024000,179.739000,1.000000,84.822105,-0.003275,-0.012393,40.512500
75%,130.606250,73.598000,92.259000,79.169000,2.961000,27.591000,200.478250,1.777000,1.225000,45.017000,...,0.959000,1.618000,51.895000,1.561000,263.379250,2.000000,84.822105,0.433000,0.503000,109.933750
max,1192.026000,690.075000,1349.351000,2834.999000,5.684000,2834.999000,1852.462000,19.773000,1.414000,764.408000,...,2.503000,3.142000,2842.617000,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,1633.433000


A first analysis can be done by checking the difference in each feature between minimun and 25% element, maximum and 75% element. This difference should be analized in relation to the standard deviation value.

F.e. check the variable DER_pt_tot, the difference between the maximum value and the valui in the percentil 75% is ~= 2800, more than 100 times the standard deviation.

#### 2.1- Remove outliers

In [24]:
from scipy import stats

In [25]:
z_train          = np.abs(stats.zscore(df_train_MV_cleaned.drop(['Prediction'], axis=1)))
df_train_cleaned = df_train_MV_cleaned[(z_train < 4).all(axis=1)]
df_train_cleaned.describe()

# In case we want to analyze outliers in test dataset
#z_test           = np.abs(stats.zscore(df_test_MV_cleaned.drop(['Prediction'], axis=1)))
#df_test_cleaned  = df_test_MV_cleaned[(z_test < 3).all(axis=1)]
#df_test_cleaned.describe()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,PRI_tau_pt,...,PRI_lep_eta,PRI_lep_phi,PRI_met,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_all_pt
count,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,...,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000,237376.000000
mean,117.834739,48.431308,78.210731,51.498414,2.387252,17.550510,144.549141,1.399114,-0.154108,37.064706,...,-0.020969,0.044199,38.825513,-0.011071,196.650440,0.928700,79.782360,-0.002967,-0.013894,62.862284
std,38.209407,32.678290,30.792343,50.925703,0.738511,18.026591,89.311476,0.705681,1.194827,17.280008,...,1.265165,1.815857,25.493060,1.812351,103.666211,0.951625,33.739601,1.381300,1.386539,78.120344
min,9.044000,0.000000,6.329000,0.000000,0.208000,0.000000,46.104000,0.203000,-1.414000,20.000000,...,-2.505000,-3.142000,0.109000,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,0.000000
25%,95.244000,19.439750,59.379750,11.839250,1.857000,2.760000,76.063000,0.890000,-1.374000,24.513000,...,-1.018000,-1.521000,20.950000,-1.577000,120.674750,0.000000,56.322000,-0.380000,-0.488000,0.000000
50%,119.106500,46.598500,73.373500,36.460000,2.505000,11.096000,115.712000,1.276000,-0.472500,31.481000,...,-0.047000,0.088000,33.966000,-0.025000,174.808000,1.000000,84.822105,-0.003275,-0.012393,37.954500
75%,129.466250,73.148000,91.037000,73.320000,2.958000,27.012250,187.099000,1.753000,1.220000,44.021000,...,0.959000,1.617000,50.120000,1.559000,251.343250,2.000000,84.822105,0.377000,0.421000,100.129250
max,332.838000,190.471000,244.387000,312.113000,5.331000,108.004000,621.165000,4.816000,1.414000,128.319000,...,2.503000,3.142000,173.242000,3.142000,712.239000,3.000000,272.812000,4.499000,3.141000,464.950000


A better approach we could consider is removing outliers in our data (in a more restricted way, removing less data) and after "cleaning" outliers in each variable by replacing them in by a deviated but less extreme value.

## 3.- Increase regresion order

Linear regression involving only given features has given us a maximum of a 75% of accuracy. In this section we create artificial features from the original data so we can create a more complex model which is able to fit better our data.

In [26]:
df_train_extended = df_train_cleaned.copy()
df_test_extended  = df_test_MV_cleaned.copy()

features = list(df_train_extended.columns)[1:]

In [27]:
# This function should generalize the behaviour of the cells below but it needs to be fixed!
#
#for i in range(len(features)):
#    for j in range(i,len(features)):
#        df_train_extended = df_train_extended.assign(aux=lambda x: (x[features[i]]*x[features[j]]))

In [28]:
df_train_extended = df_train_extended.assign(aux_1_1  =lambda x: (x[features[1]]**2))
df_train_extended = df_train_extended.assign(aux_2_2  =lambda x: (x[features[2]]**2))
df_train_extended = df_train_extended.assign(aux_3_3  =lambda x: (x[features[3]]**2))
df_train_extended = df_train_extended.assign(aux_4_4  =lambda x: (x[features[4]]**2))
df_train_extended = df_train_extended.assign(aux_5_5  =lambda x: (x[features[5]]**2))
df_train_extended = df_train_extended.assign(aux_6_6  =lambda x: (x[features[6]]**2))
df_train_extended = df_train_extended.assign(aux_7_7  =lambda x: (x[features[7]]**2))
df_train_extended = df_train_extended.assign(aux_8_8  =lambda x: (x[features[8]]**2))
df_train_extended = df_train_extended.assign(aux_9_9  =lambda x: (x[features[9]]**2))
df_train_extended = df_train_extended.assign(aux_10_10=lambda x: (x[features[10]]**2))
df_train_extended = df_train_extended.assign(aux_11_11=lambda x: (x[features[11]]**2))
df_train_extended = df_train_extended.assign(aux_12_12=lambda x: (x[features[12]]**2))
df_train_extended = df_train_extended.assign(aux_13_13=lambda x: (x[features[13]]**2))
df_train_extended = df_train_extended.assign(aux_14_14=lambda x: (x[features[14]]**2))
df_train_extended = df_train_extended.assign(aux_15_15=lambda x: (x[features[15]]**2))
df_train_extended = df_train_extended.assign(aux_16_16=lambda x: (x[features[16]]**2))
df_train_extended = df_train_extended.assign(aux_17_17=lambda x: (x[features[17]]**2))
df_train_extended = df_train_extended.assign(aux_18_18=lambda x: (x[features[18]]**2))
df_train_extended = df_train_extended.assign(aux_19_19=lambda x: (x[features[19]]**2))
df_train_extended = df_train_extended.assign(aux_20_20=lambda x: (x[features[20]]**2))
df_train_extended = df_train_extended.assign(aux_21_21=lambda x: (x[features[21]]**2))
df_train_extended = df_train_extended.assign(aux_22_22=lambda x: (x[features[22]]**2))

In [29]:
df_test_extended = df_test_extended.assign(aux_1_1  =lambda x: (x[features[1]]**2))
df_test_extended = df_test_extended.assign(aux_2_2  =lambda x: (x[features[2]]**2))
df_test_extended = df_test_extended.assign(aux_3_3  =lambda x: (x[features[3]]**2))
df_test_extended = df_test_extended.assign(aux_4_4  =lambda x: (x[features[4]]**2))
df_test_extended = df_test_extended.assign(aux_5_5  =lambda x: (x[features[5]]**2))
df_test_extended = df_test_extended.assign(aux_6_6  =lambda x: (x[features[6]]**2))
df_test_extended = df_test_extended.assign(aux_7_7  =lambda x: (x[features[7]]**2))
df_test_extended = df_test_extended.assign(aux_8_8  =lambda x: (x[features[8]]**2))
df_test_extended = df_test_extended.assign(aux_9_9  =lambda x: (x[features[9]]**2))
df_test_extended = df_test_extended.assign(aux_10_10=lambda x: (x[features[10]]**2))
df_test_extended = df_test_extended.assign(aux_11_11=lambda x: (x[features[11]]**2))
df_test_extended = df_test_extended.assign(aux_12_12=lambda x: (x[features[12]]**2))
df_test_extended = df_test_extended.assign(aux_13_13=lambda x: (x[features[13]]**2))
df_test_extended = df_test_extended.assign(aux_14_14=lambda x: (x[features[14]]**2))
df_test_extended = df_test_extended.assign(aux_15_15=lambda x: (x[features[15]]**2))
df_test_extended = df_test_extended.assign(aux_16_16=lambda x: (x[features[16]]**2))
df_test_extended = df_test_extended.assign(aux_17_17=lambda x: (x[features[17]]**2))
df_test_extended = df_test_extended.assign(aux_18_18=lambda x: (x[features[18]]**2))
df_test_extended = df_test_extended.assign(aux_19_19=lambda x: (x[features[19]]**2))
df_test_extended = df_test_extended.assign(aux_20_20=lambda x: (x[features[20]]**2))
df_test_extended = df_test_extended.assign(aux_21_21=lambda x: (x[features[21]]**2))
df_test_extended = df_test_extended.assign(aux_22_22=lambda x: (x[features[22]]**2))

## 4.- Predictor

In [35]:
from proj1_helpers import *
from implementations import *

#### 3.1- Train, generate model

In [36]:
df = df_train_extended

y  = np.array([1 if x=='s' else -1 for x in df['Prediction'].values]) # Labels 's' and 'b' to 1 and -1
tx = df.drop(['Prediction'], axis=1).values

In [37]:
w, loss = least_squares(y, tx)
loss

0.31734953607527555

#### 3.2- Test, generate predictions

In [38]:
df = df_test_extended

tx_test = df.drop(['Prediction'], axis=1).values
ids_test= df.index

In [39]:
y_pred  = predict_labels(w, tx_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)